## <span style="color:red"> **Total Points 9** </span>

# Script to illustrate the thickness estimation method by Farinotti & al 2009

> Farinotti, D., Huss, M., Bauder, A., Funk, M., & Truffer, M. (2009).
> A method to estimate the ice volume and ice-thickness distribution
> of alpine glaciers. Journal of Glaciology, 55(191), 422-430.

This code calculates the along-flow bed profile of Rhonegletscher
given 2007 data of glacier outline, surface and flowline coordinates.
For Rhonegletscher, absence of debris cover is assumed.

Written by Fabian Walter & Mauro Werder (ETH-Zurich / WSL)

*Notes*:
- you can do this exercise in any programming language you like.
- remember to look at the introductory notebook we looked at in lecture 2.

**Tasks** for you to complete are marked in bold.

**Task:** **Download and skim the manuscript by [Farinotti & al 2009](https://doi.org/10.3189/002214309788816759)**

Import packages which are needed for reading of files and plotting (hit shift+enter to execute the cell)

In [ ]:
using Pkg; Pkg.activate("."); Pkg.add("PyPlot")
using DelimitedFiles, PyPlot # DelimitedFiles is a standard lib, so no need to install

Define data file names and a function which can read Ascii-grids

In [ ]:
fname_outl = "glacier_outline_rhone_2007.txt";
fname_flowl = "flowline_rhone_2007.txt";
fname_surfdem = "surface_dem_rhone_2007.asc";
fname_mask = "glacier_mask_rhone_2007.asc";

"""
    read_asc(fl)

Read a Ascii grid https://en.wikipedia.org/wiki/Esri_grid.

In:
- fn -- file name

Out
- array (rows/first-index correspond to x-dim, columns to y-dim)
- x,y

Note: there is no need for you to understand this function, just use it.
"""
function read_asc(fl)
    T = Float64
    io = open(fl, "r")
    toT = (io,T) -> parse(T, split(readline(io))[2])
    # read header
    nc, nr, xll, yll, dx = toT(io, Int), toT(io, Int), toT(io,Float64), toT(io,Float64), toT(io,Float64)
    prop, val = split(readline(io))

    # read body
    va = Array{T}(undef, nr, nc)
    tmp = split(read(io, String))
    if length(tmp)!=nc*nr
        error("Something's wrong with the file/stream $io.  It should contain $(nc*nr) values but has $(length(tmp))")
    end
    for i=1:nr, j=1:nc
        va[i,j] = parse(T, tmp[(i-1)*nc + j])
    end
    close(io)

    x = range(xll, step=dx, length=nc)
    y = range(yll, step=dx, length=nr)

    return rotr90(va), x, y
end

Read in data:

In [ ]:
outl = readdlm(fname_outl) # glacier outline
flowl = readdlm(fname_flowl,header=true)[1] # the central flow-line
fx, fy, f_surf, f_bed, f_width = flowl[:,1], flowl[:,2], flowl[:,3], flowl[:,4], flowl[:,5]
surfdem, x, y = read_asc(fname_surfdem) # the surface DEM
mask, x, y = read_asc(fname_mask) # the mask of the glacier
mask = convert(BitMatrix, mask); # make it into an array of Boolean
dx, dy = step(x), step(y) # the grid-spacing

Calculate curvlinear, along flow coordinate, which starts
at glacier head and grows towards terminus:

In [ ]:
s = zeros(size(flowl,1));
s[1] = 0;
for i=2:length(fx)
   s[i] = s[i-1] + sqrt( (fx[i]-fx[i-1])^2 + (fy[i]-fy[i-1])^2);
end

Plot of glacier bed and surface versus the along flow coordinate.
Glacier bed is known from radar and
considered as "ground truth" for our purposes:

In [ ]:
figure()
plot(s, f_surf,"k--")
plot(s, f_bed,"k")
legend(["Glacier surface";"Glacier bed (from radar)"])
xlabel("Along-flow coordinate (m)")
ylabel("Elevation above sea level (m)")

Plot of glacier map with flow line and outline:

In [ ]:
figure(figsize=(10,10))
PyPlot.axis("equal")
X, Y = x.+y'*0, x*0 .+y';
contour(X, Y, surfdem, 1500:50:4000);
plot(outl[:,1], outl[:,2], "k*")
plot(fx, fy, "k.-")
xlabel("Easting (m in CH1903/LV03)")
ylabel("Northing (m in CH1903/LV03)")

**Task 1:** **We need to determine the ELA of the apparent
mass balance. z_ela = 2870 is a good choice. Write
a short piece of code to support this value.
Use the values given below for the apparent mass balance
gradients `gacc` and `gabl`.**
**What value should `Btilde` approximate?**

Hints:
(0) fill in the `...` in below code-block
(1) write a function which calculates `btilde` for a `z` and a `z_ela`
(2) loop over all the points on the grid, if they are within the glacier (see
    `mask`) then sum their `btilde` to get `Btilde`.

### <span style="color:red"> **Points 2** </span>

In [ ]:
gacc = 1.0*0.5*10^(-2); # apparent mass-balance gradient in accumulation zone
gabl = 1.0*0.9*10^(-2); # apparent mass-balance gradient in ablation zone
z_ela = 2870; # the "ELA" of the apparent mass-balance, i.e. the elevation where it is zero

function btilde(z, gacc, gabl, z_ela)
    if z>z_ela
        return ...
    else
        return ...
    end
end
Btilde = 0.0
for i = 1:length(surfdem)
    if ...
        Btilde = Btilde + ...
    end
end
Btilde

Calculate volume flux. This requires integrating
the apparent mass balance over the contributing
glacier surface upstream of the coordinate. Identifying
the contributing surface is difficult to automate. Here we
simply assume that the contributing surface is at higher
elevations than the flowline coordinate.

In [ ]:
function flux_at_z(z, z_ela)
    Q = 0.0
    for i = 1:length(surfdem)
        zz = surfdem[i]
        if mask[i] && zz>=z
            Q = Q + dx*dy* btilde(zz, gacc, gabl, z_ela)
        end
    end
    return Q
end
# Evaluate the volume flux for each flow-line point:
flux_at_z.(f_surf, z_ela)' ## note the ".": it applies the function to all elements of the input
                           # the ' is transpose, just to make more compact printing

**Task 2:** **Can you think of a different (better) way to do above flux calculation or is this good enough?**

Your answer (double click to type):

### <span style="color:red"> **Points 1** </span>

Now we want to apply equation (7) in Farinotti et al. (2009)

$H = \left( \frac{\bar q}{2 A}  \frac{n+2}{(C \rho g \tan(\alpha))^n} \right)^{1/(n+2)}$

We already have `qbar = Q/f_width`.  But we also need the surface slope `tan(alpha) = d z_s / d s`
(where `z_s` is the surface elevation and `s` the along flow line coordinate)

To calculate slope for each flowline grid point, we just take the mean of the forward
and backward finite difference to approximate the gradient.  This is a bit sloppy
but should be ok:

In [ ]:
slope = zeros(length(s))
for i = 2:length(s)-1
    zabove,zmid,zbelow = f_surf[i-1:i+1]
    sabove,smid,sbelow = s[i-1:i+1]
    slope[i] = 1/2*( (zmid-zabove)/(smid-sabove) + (zbelow-zmid)/(sbelow-smid))
end
# special case start and end
slope[1] = (f_surf[2]-f_surf[1])/(s[2]-s[1])
slope[end] = (f_surf[end]-f_surf[end-1])/(s[end]-s[end-1])
slope' # the ' is transpose, just to make more compact printing

**Task 3:** **Plot the slope vs flow-line coordinate, together with the flow-line surface and bed topography:**
### <span style="color:red"> **Points 1** </span>

In [ ]:
figure();
... ## plot surface
... ## plot bed
ylabel("Elevation (m)")
xlabel("Along-flow coordinate (m)")
ax = twinx()
... ## plot slope
ax.set_ylabel("Surface slope (deg)")

Calculate thickness. Note that glacier width `f_width` was read off the map manually.
We only treat points where the flux Q is non-zero (below there is no more glacier).

**Task 3:**
- **fill the gaps marked with `...`**
- **Change the value of C to investigate (and report) the sensitivity of the thickness calculation.**
- **Plot glacier surface with bed from radar and bed from calculation, i.e. modify the first figure of this notebook.**

### <span style="color:red"> **Points 3** </span>

In [ ]:
# Constants
A = 2.4*10^(-15); # ice flow factor
n = 3; # Glen's n
C = 0.25; # factor of Farinotti &al 2009
rho = 900; # density of ice
g = 9.81; # accel. due to gravity

# Loop over all points on the flow-line:
Q = zeros(length(s));
H = zeros(length(s));
qbar = zeros(length(s));
for i = 1:length(s)
    Q[i] = ...
    if Q[i]<=0
        Q[i] = qbar[i] = H[i] = 0
        continue
    end
    # Ice volume flux
    qbar[i] = ...
    # Equation (7) in Farinotti et al. (2009)
    # (if you get a DomainError, you may need an `abs`
    H[i] = ...
end
H'

In [ ]:
# Plot it!

**Task 4:** **Discuss how portable the approach by Farinotti & al 2009 is to other
glacier catchments. Is this a means to estimate glacier volumes for entire
mountain ranges, continents or the entire earth? If you challenge one or more
of the method's assumptions, then explain why this is a significant
shortcoming. Your answer should be concise and not exceed 15 sentences.**

Type your answer here:

### <span style="color:red"> **Points 2** </span>

Extra: We're still left with extrapolating our result back into 2D map-plane... but that
left as a *ungraded* exercise for the avid student.  Consult Farinotti & al 2009 on
how to go about this.